In [ ]:
import os
import pandas as pd
import subprocess

# generate an empty dataframe to be filled with each sample sequenced in the experiment
sample_df = pd.DataFrame()

# assign the file path to the experiments sequencing data
directory = "/put/path/to/directory/containing/bam/files/here"

# iterate over each file in the directory, remove duplicate names, and populate lists with a minimal sample name as well as the cell counts
file_list = []
sample_list = []
cell_count_list = []

for filename in os.listdir(directory):
    if "sorted.markdup.bam.bai" in filename:
        cell_count_list.append(1)
        file_list.append(filename[:-3])
        sample_list.append(filename[:5]) # CHANGEME: QC this step to be sure you are getting what you want

# fill the sample data frame in with the list generated from the experiment directory
sample_df["file"] = sc_file_list
sample_df["sample"] = sc_sample_list
sample_df["cell_count"] = sc_cell_count_list

# randomly take a subset of the samples sent to sequencing in order to save CPU and time
test_df = sample_df.sample(frac=0.5)

# read the data into permanent storage and optionally check the samples in the test dataframe
test_df.to_csv("/put/path/to/directory/containing/experiment/data/here")
test_df

In [ ]:
# terminal command to get chromosome specific reads
# samtools view -c -F 0x4 /file chr1 (chr2, chrX, chrY)
x = 1
while x <= 24:
    if x == 23:
        chr = "chrX"
    elif x == 24:
        chr = "chrY"
    else:
        chr = "chr" + str(x)

    chr_list = []

    for filename in test_df["file"]:
        chr_reads = subprocess.run(["samtools", "view", "-c", "-F", "0x4", directory + filename, chr], stdout=subprocess.PIPE, text=True)
        chr_list.append(chr_reads.stdout[:-1])

    test_df[chr] = chr_list
    x += 1

# optionally check the read counts mapping to each chromosome of each sample in the dataframe 
test_df #;

In [ ]:
# terminal command to get total reads
# samtools view -c -F 0x4 /file
chr_list = []

for filename in test_df["file"]:
        chr_reads = subprocess.run(["samtools", "view", "-c", "-F", "0x4", directory + filename], stdout=subprocess.PIPE, text=True)
        chr_list.append(chr_reads.stdout[:-1])
        
test_df["total_reads"] = chr_list

# convert the numberic collumns in the dataframe to integers
test_df["total_reads"] = test_df["total_reads"].astype(int)
test_df["cell_count"] = test_df["cell_count"].astype(int)

# optionally check the read counts mapping to each chromosome of each sample in the dataframe 
test_df #;

In [ ]:
# create a collumn corresponding to the percentage of reads mapping to each chromosome
total_read_list = [put, total, reads, here] # FIXME: get rid of the manual imput of total reads for each sample

x = 1
while x <= 24:
    if x == 23:
        chr = "chrX"
    elif x == 24:
        chr = "chrY"
    else:
        chr = "chr" + str(x)

    # convert the numeric collumns to integers
    test_df[chr] = test_df[chr].astype(int)

    chr_list = []
    y = 0 # references back to the index of the total read list

    for count in test_df[chr]:
        chr_percent = (count / total_read_list[y]) * 100
        chr_list.append(chr_percent)
        y += 1
    
    test_df[chr + "_percent"] = chr_list
    x += 1

# optionally check the percentage of total reads mapping to each chromosome of each sample in the dataframe 
test_df #;   

In [ ]:
# use this code snippet to rearrange the data
# opened the full output with a text editor and saved file
x = 1
while x <= 24:
    if x == 23:
        chr = "chrX_percent"
    elif x == 24:
        chr = "chrY_percent"
    else:
        chr = "chr" + str(x) + "_percent"

    for index in test_df.index:
        if len(chr) == 12:
            print((test_df["sample"][index]), test_df["cell_count"][index], chr[3:4], test_df[chr][index])
        elif len(chr) == 13:
            print((test_df["sample"][index]), test_df["cell_count"][index], chr[3:5], test_df[chr][index])
    
    x+=1